In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [2]:
data_path = os.path.join("model_data")

actions = []

for i in range(65,91):
    actions.append(chr(i))

sequences_no = 60

sequences_length = 30

In [3]:
actions = np.array(actions)

In [39]:
# actions = np.array(['A','B'])

In [4]:
actions

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype='<U1')

In [5]:
data_path

'model_data'

In [6]:
label_map = {}
i = 0
for num in actions:
    label_map[num] = i
    i += 1


In [7]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in range(sequences_no):
        window = []
        for frame_num in range(sequences_length):
            path = os.path.join(data_path,str(action),str(sequence),"{}.npy".format(frame_num))
            res = np.load(path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
X = np.array(sequences)

In [10]:
X.shape

(1560, 30, 126)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [14]:
X_train

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.22881138e-01,  3.71437967e-01, -3.40468027e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.23143145e-01,  3.71372759e-01, -3.37136276e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.23130465e-01,  3.71641994e-01, -3.37862186e-02],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.22770616e-01,  3.73515278e-01, -3.49466354e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.23480240e-01,  3.72662961e-01, -3.18816490e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.22698271e-01,  3.73246878e-01, -3.47904339e-02]],

       [[ 6.37011707e-01,  5.01522601e-01,  1.18516638e-07, ...,
          5.49078763e-01,  6.80177093e-01, -4.39829044e-02],
        [ 6.37052298e-01,  5.01438737e-01,  1.18709238e-07, ...,
          5.48539400e-01,  6.75391078e

Traning Neural Network (LSTM)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [16]:
model = Sequential()

model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(BatchNormalization())

model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=False, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))

model.add(Dense(32, activation = 'relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\agarw\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)

In [18]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['categorical_accuracy'])

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 128)        │       130,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 751,162 (2.87 MB)

 Trainable params: 750,394 (2.86 MB)

 Non-trainable params: 768 (3.00 KB)

In [139]:
model.fit(X_train,y_train, epochs=1)

44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - categorical_accuracy: 0.9971 - loss: 0.0161


In [140]:
res = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [150]:
actions[np.argmax(res[2])]

'H'

In [154]:
y_pred = []
for i in res:
    print(actions[np.argmax(i)],end=" ")
    y_pred.append(np.argmax(i))

A W H T R W J E R V F E M K X F T X A J S U F U V M B T D S X Q I T Y Q U Z Y H T L H U C C B X R W T X A E E B M T W Z G M H S M C F P R W J N L K F V U T N Q H F Z O N T F P L G D J Q G N R N G F F P P S V Y X G N H I I I O H G Z B T L T O F K N T C D K I C H H O Q K I N T E S O S T T I E T K V R Q S R A E X 

In [155]:
y_true = []
for i in y_test:
    print(actions[np.argmax(i)],end=" ")
    y_true.append(np.argmax(i))

A W H T R W J E R V F E M K X F T X A J S U F U V M B T D S X Q I A Y Q U Z Y H A L H U C C B X R W T X A E E B M A W Z G M H S M C F P R W J N L K F V U T N Q H F Z O N A F P L G D J Q G N R N G F F P P S V Y X G N H I I I O H G Z B A L T O F K N T C D K I C H H O Q K I N T E S O S T T I E T K V R Q S R A E X 

In [60]:
model.save('modek.keras')

In [156]:
print("Actual values:",y_true)
print("Predicted Value:",y_pred)

Actual values: [0, 22, 7, 19, 17, 22, 9, 4, 17, 21, 5, 4, 12, 10, 23, 5, 19, 23, 0, 9, 18, 20, 5, 20, 21, 12, 1, 19, 3, 18, 23, 16, 8, 0, 24, 16, 20, 25, 24, 7, 0, 11, 7, 20, 2, 2, 1, 23, 17, 22, 19, 23, 0, 4, 4, 1, 12, 0, 22, 25, 6, 12, 7, 18, 12, 2, 5, 15, 17, 22, 9, 13, 11, 10, 5, 21, 20, 19, 13, 16, 7, 5, 25, 14, 13, 0, 5, 15, 11, 6, 3, 9, 16, 6, 13, 17, 13, 6, 5, 5, 15, 15, 18, 21, 24, 23, 6, 13, 7, 8, 8, 8, 14, 7, 6, 25, 1, 0, 11, 19, 14, 5, 10, 13, 19, 2, 3, 10, 8, 2, 7, 7, 14, 16, 10, 8, 13, 19, 4, 18, 14, 18, 19, 19, 8, 4, 19, 10, 21, 17, 16, 18, 17, 0, 4, 23]
Predicted Value: [0, 22, 7, 19, 17, 22, 9, 4, 17, 21, 5, 4, 12, 10, 23, 5, 19, 23, 0, 9, 18, 20, 5, 20, 21, 12, 1, 19, 3, 18, 23, 16, 8, 19, 24, 16, 20, 25, 24, 7, 19, 11, 7, 20, 2, 2, 1, 23, 17, 22, 19, 23, 0, 4, 4, 1, 12, 19, 22, 25, 6, 12, 7, 18, 12, 2, 5, 15, 17, 22, 9, 13, 11, 10, 5, 21, 20, 19, 13, 16, 7, 5, 25, 14, 13, 19, 5, 15, 11, 6, 3, 9, 16, 6, 13, 17, 13, 6, 5, 5, 15, 15, 18, 21, 24, 23, 6, 13, 7, 8, 8, 8, 1

In [144]:
from sklearn.metrics import classification_report
print("Classification Report: \n", classification_report(actions[y_true], actions[y_pred]))

Classification Report: 
               precision    recall  f1-score   support

           A       1.00      0.44      0.62         9
           B       1.00      1.00      1.00         4
           C       1.00      1.00      1.00         5
           D       1.00      1.00      1.00         3
           E       1.00      1.00      1.00         7
           F       1.00      1.00      1.00        10
           G       1.00      1.00      1.00         6
           H       1.00      1.00      1.00         9
           I       1.00      1.00      1.00         7
           J       1.00      1.00      1.00         4
           K       1.00      1.00      1.00         6
           L       1.00      1.00      1.00         4
           M       1.00      1.00      1.00         5
           N       1.00      1.00      1.00         8
           O       1.00      1.00      1.00         5
           P       1.00      1.00      1.00         4
           Q       1.00      1.00      1.00         6
  